In [36]:
import pandas as pd
import numpy as np
import requests
import os

In [37]:
if not os.path.exists('ml-20m'):

    import urllib.request
    urllib.request.urlretrieve('http://files.grouplens.org/datasets/movielens/ml-20m.zip', 'ml-20m.zip')

    import zipfile
    with zipfile.ZipFile('ml-20m.zip',"r") as zip_ref:
        zip_ref.extractall()
        
    os.remove('ml-20m.zip')

In [38]:
movies = pd.read_csv('ml-20m/movies.csv')
movies = movies[(movies['genres'] != '(no genres listed)') & ('IMAX' not in movies['genres'])]
#movies.head()

In [39]:
links = pd.read_csv('ml-20m/links.csv')
links.dropna(inplace=True)
links.tmdbId = links.tmdbId.astype(int)
#links.head()

In [44]:
movies = pd.merge(movies, links, how='right', on=['movieId'])
movies.head()

movies_sample = movies.sample(n=3000, random_state=1)
movies_sample.head()

,movieId,title,genres,imdbId_x,tmdbId_x,imdbId_y,tmdbId_y
24024,115151,Plastic (2014),Action|Crime,2556874,208869,2556874,208869
20937,102956,Take Me Home (2011),Comedy|Romance,1261954,85431,1261954,85431
24367,116570,The Youngest Profession (1943),Comedy|Romance,36554,108852,36554,108852
22149,107372,Happy Family (2010),Comedy,1455810,42892,1455810,42892
619,628,Primal Fear (1996),Crime|Drama|Mystery|Thriller,117381,1592,117381,1592


In [28]:
from collections import Set

movies_sample = movies.sample(n=3000, random_state=1)
movies_sample.head()

def calc_existing_genres(movies):
    genres_set = set()
    for genre_list in movies['genres']:
        for genre in genre_list.split('|'):
            genres_set.add(genre)
    return genres_set
        
calc_existing_genres(movies_sample)

{'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

In [4]:
tags = pd.read_csv('ml-20m/tags.csv')
tags.head()

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,1240597180
1,65,208,dark hero,1368150078
2,65,353,dark hero,1368150079
3,65,521,noir thriller,1368149983
4,65,592,dark hero,1368150078


In [6]:
posters_base_url = 'https://api.themoviedb.org/3/movie/{}/images?api_key=c4d5a44a2b7b36e4d40d0d1e713a8fee'

import requests
response = requests.get(posters_base_url.format(862))
posters = response.json()['posters']
posters

[{'aspect_ratio': 0.6666666666666666,
  'file_path': '/uMZqKhT4YA6mqo2yczoznv7IDmv.jpg',
  'height': 1500,
  'iso_639_1': 'en',
  'vote_average': 6.12,
  'vote_count': 19,
  'width': 1000},
 {'aspect_ratio': 0.6666666666666666,
  'file_path': '/agy8DheVu5zpQFbXfAdvYivF2FU.jpg',
  'height': 1500,
  'iso_639_1': 'es',
  'vote_average': 5.618,
  'vote_count': 41,
  'width': 1000},
 {'aspect_ratio': 0.6666666666666666,
  'file_path': '/wLz9NiVCoABanSQEAXciYv3qjSc.jpg',
  'height': 2100,
  'iso_639_1': 'en',
  'vote_average': 5.456,
  'vote_count': 5,
  'width': 1400},
 {'aspect_ratio': 0.6666666666666666,
  'file_path': '/9Ijv9yyg32mlQbJDYbOoKlWlWbp.jpg',
  'height': 1500,
  'iso_639_1': 'en',
  'vote_average': 5.384,
  'vote_count': 2,
  'width': 1000},
 {'aspect_ratio': 0.6666666666666666,
  'file_path': '/rhIRbceoE9lR4veEXuwCC2wARtG.jpg',
  'height': 1500,
  'iso_639_1': 'en',
  'vote_average': 5.322,
  'vote_count': 5,
  'width': 1000},
 {'aspect_ratio': 0.6666666666666666,
  'file_pat

In [51]:
posters_base_url = 'https://api.themoviedb.org/3/movie/{}/images?api_key=c4d5a44a2b7b36e4d40d0d1e713a8fee'

for index, row in movies_sample.iteritems():
    print(row)
    response = requests.get(posters_base_url.format(row.tmdbId))
    break
    
response

24024    115151
20937    102956
24367    116570
22149    107372
619         628
10272     34624
26143    127280
3994       4125
21201    103819
22840    110026
2569       2689
19171     95969
25210    120789
6881       7034
13206     64930
7437       7787
19347     96669
15334     78528
5375       5510
5071       5206
20303    100310
22946    110415
4496       4628
19777     98337
22537    108893
3059       3181
20706    102025
25083    119950
3128       3251
13816     69521
          ...  
23881    114453
19295     96510
17284     87647
7439       7790
7863       8589
13169     64519
3119       3242
24169    115826
8842      26504
69           70
9938      32825
22779    109796
3215       3338
11316     48520
19185     96026
26636    130466
22622    109237
20939    102963
25117    120222
15745     80328
12914     61406
7312       7492
13296     65631
4343       4475
13745     69217
2314       2431
5433       5569
8902      26606
12456     58404
21819    106144
Name: movieId, Length: 3

AttributeError: 'Series' object has no attribute 'tmdbId'

In [8]:
poster_base_url = 'https://image.tmdb.org/t/p/w300{}'

poster_url = poster_base_url.format(posters[0]['file_path'])
poster_url

'https://image.tmdb.org/t/p/w300/uMZqKhT4YA6mqo2yczoznv7IDmv.jpg'

In [11]:
import urllib.request

urllib.request.urlretrieve(poster_url, './test.jpg')

('./test.jpg', <http.client.HTTPMessage at 0x276f1fecd30>)